In [ ]:
import os
import time
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import requests
import pandas as pd

# === Configuration Parameters ===
GAME_NAMES = ["Slay the Spire"]  # List of game names to process
START_DATE_STR = "2025-01-01"
END_DATE_STR = "2025-05-20"
LANGUAGE = "english"  # e.g., english, all
MARKET = "us"  # Country code for review filtering
SAMPLE_SIZE = 1000  # Number of unique user reviews to sample

# === Helper Functions ===

def load_app_list():
    """
    Load the full Steam app list (appid, name).
    """
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.json().get("applist", {}).get("apps", [])


def find_appid_by_name(apps, name):
    """
    Find the app ID by exact or case-insensitive name match.
    """
    for app in apps:
        if app.get("name", "").lower() == name.lower():
            return app.get("appid")
    for app in apps:
        if name.lower() in app.get("name", "").lower():
            return app.get("appid")
    return None


def get_reviews(app_id, country, language, per_page, start_date, end_date, max_reviews):
    """
    Fetch up to max_reviews reviews for the given app_id between start_date and end_date,
    filtering by language and country, via paged requests with User-Agent header.
    """
    start_ts = int(start_date.timestamp()) if start_date else None
    end_ts = int(end_date.timestamp()) if end_date else None
    reviews = []
    cursor = "*"
    headers = {"User-Agent": "Mozilla/5.0"}

    while len(reviews) < max_reviews:
        params = {
            "json": 1,
            "filter": "recent",
            "language": language,
            "purchase_type": "all",
            "cc": country,
            "num_per_page": per_page,
            "cursor": cursor
        }
        resp = requests.get(
            f"https://store.steampowered.com/appreviews/{app_id}",
            params=params,
            headers=headers
        )
        resp.raise_for_status()
        data = resp.json()
        batch = data.get("reviews", [])
        if not batch:
            break
        for r in batch:
            ts = r.get("timestamp_created", 0)
            if start_ts and ts < start_ts:
                return reviews
            if end_ts and ts > end_ts:
                continue
            reviews.append(r)
            if len(reviews) >= max_reviews:
                break
        cursor = data.get("cursor", cursor)
        time.sleep(0.2)
    return reviews


def is_profile_public(api_key, steam_id):
    """
    Returns True if the Steam profile is public.
    """
    url = "https://api.steampowered.com/ISteamUser/GetPlayerSummaries/v2/"
    params = {"key": api_key, "steamids": steam_id}
    try:
        resp = requests.get(url, params=params)
        resp.raise_for_status()
        players = resp.json().get("response", {}).get("players", [])
        return bool(players and players[0].get("communityvisibilitystate") == 3)
    except requests.exceptions.RequestException:
        return False


def get_owned_games(api_key, steam_id):
    """
    Fetch owned games list; returns [] if profile is private.
    """
    url = "https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/"
    params = {
        "key": api_key,
        "steamid": steam_id,
        "include_appinfo": 1,
        "include_played_free_games": 1,
        "format": "json"
    }
    try:
        resp = requests.get(url, params=params)
        if resp.status_code in (403, 401):
            return []
        resp.raise_for_status()
        return resp.json().get("response", {}).get("games", [])
    except requests.exceptions.RequestException:
        return []


def main():
    # Load API key
    load_dotenv(find_dotenv(), override=True)
    api_key = os.getenv("STEAM_API_KEY", "").strip()
    if not api_key:
        raise RuntimeError("Missing STEAM_API_KEY in .env")

    # Load app list for name-to-id resolution
    print("Loading Steam app list...")
    apps = load_app_list()
    start_date = datetime.strptime(START_DATE_STR, "%Y-%m-%d")
    end_date = datetime.strptime(END_DATE_STR, "%Y-%m-%d")

    for game_name in GAME_NAMES:
        print(f"\nProcessing '{game_name}'...")
        appid = find_appid_by_name(apps, game_name)
        if not appid:
            print(f"AppID not found for '{game_name}', skipping.")
            continue
        print(f"AppID for '{game_name}': {appid}")

        # Fetch reviews
        reviews = get_reviews(
            app_id=appid,
            country=MARKET,
            language=LANGUAGE,
            per_page=100,
            start_date=start_date,
            end_date=end_date,
            max_reviews=SAMPLE_SIZE
        )
        print(f"Collected {len(reviews)} reviews for '{game_name}'")

        # Extract unique public profiles
        steamids = []
        for rev in reviews:
            sid = rev.get("author", {}).get("steamid")
            if sid and sid not in steamids and is_profile_public(api_key, sid):
                steamids.append(sid)
            if len(steamids) >= SAMPLE_SIZE:
                break
        print(f"Sampling {len(steamids)} public profiles")

        # Build owned-games matrix
        rows = []
        for sid in steamids:
            games = get_owned_games(api_key, sid)
            row = {"steamid": sid}
            for g in games:
                row[g.get("name")] = g.get("playtime_forever", 0) / 60
            rows.append(row)

        # Create DataFrame and export
        df = pd.DataFrame(rows).fillna(0) if rows else pd.DataFrame()
        if not df.empty and "steamid" in df.columns:
            cols = ["steamid"] + [c for c in df.columns if c != "steamid"]
            df = df[cols]
        output_file = f"{game_name.replace(' ', '_')}_other_games_played.xlsx"
        df.to_excel(output_file, index=False)
        print(f"Exported data for {len(df)} users to {output_file}")

if __name__ == "__main__":
    main()
